In [1]:
import spacy
nlp = spacy.load("en_core_web_sm")

import re
e1pt = re.compile('<e1>.*</e1>')
e2pt = re.compile('<e2>.*</e2>')
E1LABEL = 'E1LABEL'
E2LABEL = 'E2LABEL'

from nltk.corpus import wordnet as wn

import tensorflow as tf
import numpy as np
from matplotlib import pyplot as plt
from sklearn.preprocessing import LabelEncoder

In [2]:
WORD_DIM = 300

In [3]:
# example
    # sentence
    # entity_idx
    # context_idx
    # hypernyms
    # words
    # label

In [4]:
if WORD_DIM == 50:
    with open('./glove.twitter.27B.50d.txt', 'r') as rf:
        word_dict = {}
        lines = rf.readlines()
        count = 0
        print('length is {}'.format(len(lines)))
        for line in lines:
            ls = line[:-1].split(' ')
            word_dict[ls[0]] = [float(item) for item in ls[1:]]
            if count % 10000 == 0:
                print('\rfinished {}'.format(count), end='', flush=True)
            count += 1
    #         break
        print('')
    print(len(word_dict))
elif WORD_DIM == 300:
    with open('../wiki6b/glove.6B.300d.txt', 'r') as rf:
        word_dict = {}
        lines = rf.readlines()
        count = 0
        print('length is {}'.format(len(lines)))
        for line in lines:
            ls = line[:-1].split(' ')
            word_dict[ls[0]] = [float(item) for item in ls[1:]]
            if count % 10000 == 0:
                print('\rfinished {}'.format(count), end='', flush=True)
            count += 1
    #         break
        print('')
    print(len(word_dict))
else:
    print('word_dim not right')

length is 400000
finished 390000
400000


In [5]:
# read text file
def read_txt(path):
    with open(path, 'r') as rf:
        lines = rf.readlines()
        li = 0
        sentences = []
        relations = []
        for i in range(0,len(lines), 4):
            sentences.append(lines[i])
            relations.append(lines[i+1])
    print(len(sentences), len(relations))
    return sentences, relations

train_sentences, train_relations = read_txt('./raw/TRAIN_FILE.TXT')
test_sentences, test_relations = read_txt('./raw/TEST_FILE_FULL.TXT')

8000 8000
2717 2717


In [6]:
# process labels
# RELATION_NUM = 19
OTHER_LABEL = 'Other'
RELATION_NUM = 10
if RELATION_NUM == 10:
    for i in range(len(train_relations)):
        if train_relations[i] == 'Other\n':
            train_relations[i] = train_relations[i][:-1]
            continue
        train_relations[i] = train_relations[i][:-8]
    for i in range(len(test_relations)):
        if test_relations[i] == 'Other\n':
            test_relations[i] = test_relations[i][:-1]
            continue
        test_relations[i] = test_relations[i][:-8]
elif RELATION_NUM == 19:
    for i in range(len(train_relations)):
        train_relations[i] = train_relations[i][:-1]
    for i in range(len(test_relations)):
        test_relations[i] = test_relations[i][:-1]
    
unique_set = set(test_relations + train_relations)
unique_set.remove(OTHER_LABEL)
unique_relations = list(unique_set)
unique_relations.append(OTHER_LABEL)
print(len(train_relations), train_relations[:3])
print(len(test_relations), test_relations[:3])
print(len(unique_relations), unique_relations)

8000 ['Component-Whole', 'Other', 'Instrument-Agency']
2717 ['Message-Topic', 'Product-Producer', 'Instrument-Agency']
10 ['Entity-Destination', 'Entity-Origin', 'Content-Container', 'Message-Topic', 'Product-Producer', 'Member-Collection', 'Cause-Effect', 'Instrument-Agency', 'Component-Whole', 'Other']


In [7]:
# unique_relations = ['Message-Topic', 'Instrument-Agency', 'Product-Producer', 'Content-Container', 'Entity-Origin', 'Component-Whole', 'Entity-Destination', 'Member-Collection', 'Cause-Effect',  'Other']
rel2lb = {}
lb2rel = {}
for i,item in enumerate(unique_relations):
    rel2lb[item] = i
    lb2rel[i] = item
train_labels = [rel2lb[item] for item in train_relations]
test_labels = [rel2lb[item] for item in test_relations]
print(len(train_labels), len(test_labels), test_labels[:3])

8000 2717 [3, 4, 7]


In [8]:
def process_sentence(sentence):
    # regex replace
    sentence = re.sub('\d*\t\"', "", sentence)
    sentence = sentence[:-2]
    e1 = e1pt.search(sentence)
    sentence = re.sub(e1pt, E1LABEL, sentence)
    e2 = e2pt.search(sentence)
    sentence = re.sub(e2pt, E2LABEL, sentence)
    
    # split word
    words= []
    doc = nlp(sentence)
    for token in doc:
        words.append(token.text)
    
    return sentence, (e1.group(0)[4:-5],e2.group(0)[4:-5]), words

def process_sentence_group(org_sentences):
    entities = []
    words_list = []
    processed_sentences = []
    for i in range(len(org_sentences)):
        sentence = org_sentences[i]
        sentence, entity, words =  process_sentence(sentence)
        entities.append(entity)
        words_list.append(words)
        processed_sentences.append(sentence)
        if (i%100 == 0):
            print('\rfinished {}'.format(i),flush=True, end='')
    print('')
    return processed_sentences, entities, words_list

train_processed, train_entities, train_words = process_sentence_group(train_sentences)
test_processed, test_entities, test_words = process_sentence_group(test_sentences)

finished 7900
finished 2700


In [9]:
print(train_processed[:3])
print(train_entities[:3])
print(train_words[:3])

['The system as described above has its greatest application in an arrayed E1LABEL of antenna E2LABEL.', 'The E1LABEL was carefully wrapped and bound into the E2LABEL by means of a cord.', 'The E1LABEL of a keygen uses a E2LABEL to look at the raw assembly code.']
[('configuration', 'elements'), ('child', 'cradle'), ('author', 'disassembler')]
[['The', 'system', 'as', 'described', 'above', 'has', 'its', 'greatest', 'application', 'in', 'an', 'arrayed', 'E1LABEL', 'of', 'antenna', 'E2LABEL', '.'], ['The', 'E1LABEL', 'was', 'carefully', 'wrapped', 'and', 'bound', 'into', 'the', 'E2LABEL', 'by', 'means', 'of', 'a', 'cord', '.'], ['The', 'E1LABEL', 'of', 'a', 'keygen', 'uses', 'a', 'E2LABEL', 'to', 'look', 'at', 'the', 'raw', 'assembly', 'code', '.']]


In [10]:
def lexical_features(org_words):
    context_words = []
    word_idxs = []
    count = 0
    for each in org_words:
        w1 = ''
        w2 = ''
        w3 = ''
        w4 = ''
        idx1 = 0
        idx2 = 0
        for i in range(len(each)):
            if each[i] == E1LABEL:
                w1 = each[i-1] if i-1>=0 else '<padding_word>'
                w2 = each[i+1] if i+1<len(each) else '<padding_word>'
                idx1 = i
            if each[i] == E2LABEL:
                w3 = each[i-1] if i-1>=0 else '<padding_word>'
                w4 = each[i+1] if i+1<len(each) else '<padding_word>'
                idx2 = i
        context_words.append((w1, w2, w3, w4))
        word_idxs.append([idx1, idx2])
        if count%100 == 0:
            print('\rfinished {}'.format(count), flush=True, end='')
        count += 1
    print('')
    return context_words, word_idxs

train_context,train_idxs = lexical_features(train_words)
test_context,test_idxs = lexical_features(test_words)

finished 7900
finished 2700


In [11]:
print(train_context[:3])
print(train_idxs[:3])

[('arrayed', 'of', 'antenna', '.'), ('The', 'was', 'the', 'by'), ('The', 'of', 'a', 'to')]
[[12, 15], [1, 9], [1, 7]]


In [12]:
def replace_word(org_list, org_word_list):
    new_list = []
    for i in range(len(org_list)):
        e1 = (nlp(org_list[i][0])[-1].text).lower()
        e2 = (nlp(org_list[i][1])[-1].text).lower()
        new_list.append((e1,e2))
        for j in range(len(org_word_list[i])):
            if org_word_list[i][j] == E1LABEL:
                org_word_list[i][j] = e1
            if org_word_list[i][j] == E2LABEL:
                org_word_list[i][j] = e2
            org_word_list[i][j] = org_word_list[i][j].lower()
        if i%100 == 0:
            print('\rfinished {}'.format(i), flush=True, end='')
    print('')
    return new_list

train_entities = replace_word(train_entities, train_words)
test_entities = replace_word(test_entities, test_words)
print(train_entities[:3])
print(train_words[:3])

finished 7900
finished 2700
[('configuration', 'elements'), ('child', 'cradle'), ('author', 'disassembler')]
[['the', 'system', 'as', 'described', 'above', 'has', 'its', 'greatest', 'application', 'in', 'an', 'arrayed', 'configuration', 'of', 'antenna', 'elements', '.'], ['the', 'child', 'was', 'carefully', 'wrapped', 'and', 'bound', 'into', 'the', 'cradle', 'by', 'means', 'of', 'a', 'cord', '.'], ['the', 'author', 'of', 'a', 'keygen', 'uses', 'a', 'disassembler', 'to', 'look', 'at', 'the', 'raw', 'assembly', 'code', '.']]


In [13]:
# def get_hypernyms(org_entities):
#     hypernyms = []
#     for i in range(len(org_entities)):
#         h1 = ''
#         h2 = ''
#         temp = wn.synsets(org_entities[i][0])
#         if len(temp) > 0:
#             t2 = temp[0].hypernyms()
#             if len(t2) > 0:
#                 h1 = t2[0].lemmas()[0].name()
#         temp = wn.synsets(org_entities[i][1])
#         if len(temp) > 0:
#             t2 = temp[0].hypernyms()
#             if len(t2) > 0:
#                 h2 = t2[0].lemmas()[0].name()
        
#         h1 = h1.split('_')[-1]
#         h2 = h2.split('_')[-1]
#         hypernyms.append((h1, h2))
#         if i%100 == 0:
#             print('\rfinished {}'.format(i), flush=True, end='')
#     print('')
#     return hypernyms

# train_hypernyms = get_hypernyms(train_entities)
# test_hypernyms = get_hypernyms(test_entities)
# print(train_hypernyms[:3])

In [14]:
unique_words = list(set([w for item in train_words+test_words for w in item]))
unique_words.append('<unknown_word>')
unique_words.append('<padding_word>')
unknown_idx = len(unique_words) - 2
padding_idx = len(unique_words) - 1

In [15]:
word2idx = {}
idx2word = {}
for i,w in enumerate(unique_words):
    word2idx[w] = i
    idx2word[i] = w
print(len(word2idx), len(idx2word))

22549 22549


In [16]:
not_in_count = 0
not_in_words = []
def get_word_embed(unique_words, word_dict):
    global not_in_count, not_in_words
    word_embed = np.zeros([WORD_DIM, len(unique_words)], dtype= 'float32')
    for i, w in enumerate(unique_words):
        if w not in word_dict:
            not_in_count += 1
            not_in_words.append(w)
        word_embed[:, i] = word_dict[w] if w in word_dict else np.random.normal(0,1,WORD_DIM)
    word_embed[:,-1] = np.zeros(WORD_DIM, dtype='float')
    return word_embed
word_embed = get_word_embed(unique_words, word_dict)
print(word_embed[:3,:3])
print(not_in_count, not_in_words[:3],len(unique_words))

[[-0.14945  -0.026645  0.21009 ]
 [-0.22366  -0.49923  -0.38972 ]
 [ 0.23448  -0.28152  -0.073587]]
1128 ['birdtown', 'pedologic', 'meinie'] 22549


In [17]:
MAX_LEN = 0
for item in train_words + test_words:
    if len(item) > MAX_LEN:
        MAX_LEN = len(item)
print(MAX_LEN)

97


In [18]:
data_config = {
    'WORD_DIM': WORD_DIM,
    'MAX_LEN': MAX_LEN,
    'RELATION_NUM': RELATION_NUM
}
data_config

{'WORD_DIM': 300, 'MAX_LEN': 97, 'RELATION_NUM': 10}

In [19]:
# example
    # sentence
    # entity_idx
    # entity_words
    # context_words
    # hypernyms
    # words
    # label
train_iter = zip(train_sentences, train_idxs, train_words, train_labels)
test_iter = zip(test_sentences, test_idxs, test_words, test_labels)


def transform_and_write_tf(data):
    tf_ex_ser = []
    for i,each in enumerate(data):
        sentence = each[0]
        idxs = [item for item in each[1]]
        words = each[2]
        label = each[3]
        words = [word2idx[item] if item in word2idx else unknown_idx for item in words]
        f1 = tf.train.Feature(bytes_list=tf.train.BytesList(value=[sentence.encode('utf-8')]))
        f2 = tf.train.Feature(int64_list=tf.train.Int64List(value=idxs))
        f3 = tf.train.Feature(int64_list=tf.train.Int64List(value=words))
        f4 = tf.train.Feature(int64_list=tf.train.Int64List(value=[label]))
        feature = {
            'sentece': f1,
            'idxs': f2,
            'words':f3,
            'label':f4
        }

        example = tf.train.Example(features=tf.train.Features(feature=feature)).SerializeToString()
        tf_ex_ser.append(example)
        if (i+1) % 100 == 0:
            print('\rfinished {}'.format(i+1), end='', flush=True)
    print('')
    return tf_ex_ser


train_tf = transform_and_write_tf(train_iter)
test_tf = transform_and_write_tf(test_iter)
#     print(sentence)
#     print(idxs)
#     print(lexical)
#     print(words)


finished 8000
finished 2700


In [20]:
base_dir = './data/'

In [21]:
record_file = base_dir + 'train.tfrecords'
with tf.io.TFRecordWriter(record_file) as writer:
    for example in train_tf:
        writer.write(example)

record_file = base_dir + 'test.tfrecords'
with tf.io.TFRecordWriter(record_file) as writer:
    for example in test_tf:
        writer.write(example)
print('finished')

finished


In [22]:
import pickle
pickle.dump(word2idx, open(base_dir + 'word2idx.dict', 'wb'))
pickle.dump(idx2word, open(base_dir + 'idx2word.dict', 'wb'))
pickle.dump(lb2rel, open(base_dir + 'lb2rel.dict', 'wb'))
pickle.dump(rel2lb, open(base_dir + 'rel2lb.dict', 'wb'))
pickle.dump(word_embed, open(base_dir + 'word_embed','wb'))
pickle.dump(unique_relations, open(base_dir + 'unique_relations', 'wb'))
pickle.dump(data_config, open(base_dir + 'data_config.dict', 'wb'))

In [23]:
pickle.dump(data_config, open(base_dir + 'data_config.dict', 'wb'))
print()